In [121]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import date

In [122]:
team_urls = pd.read_csv('Spiele_Links.csv')
idx = 3
print(len(team_urls))
url = team_urls['url'][idx]
response = requests.get(url)

9


In [123]:
#BeautifulSoup HTML-Dokument aus dem Quelltext parsen
soup= BeautifulSoup(response.text, 'html.parser')

In [124]:
#Tabelle aus dem HTML-Dokument extrahieren
table = soup.find('div', {"id":'id-team-matchplan-table'})
#print(table)

In [133]:
team=team_urls['team'][idx]
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    dates = list()
    times = list()
    teams = list()
    for i in range(1,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        date = rows[i].find('td', class_='column-date') #rows[i].text
        cleanString = re.sub(r'\ \|',',', date.text)
        cleanString = re.sub(r' ', '', cleanString)
        #removedSpace = re.sub(r'\t', '', removedSpace)
        splitted= cleanString.split(',')
        date_split = splitted[1].split('.')
        datum = datetime.date(int(date_split[2])+2000, int(date_split[1]), int(date_split[0]))
        print(datum.isocalendar())
        dates.append(splitted[1])
        print(splitted[1])
        times.append(splitted[2])
        teams.append(team)
        
df = pd.DataFrame(list(zip(dates,times,teams)),columns=['Datum','Zeit','Team'])
df

(2023, 16, 5)
21.04.23
(2023, 17, 5)
28.04.23
(2023, 18, 5)
05.05.23
(2023, 19, 5)
12.05.23
(2023, 20, 5)
19.05.23
(2023, 23, 5)
09.06.23
(2023, 24, 5)
16.06.23


,Datum,Zeit,Team
0,21.04.23,18:30,B-J-innen
1,28.04.23,18:30,B-J-innen
2,05.05.23,18:30,B-J-innen
3,12.05.23,18:30,B-J-innen
4,19.05.23,18:30,B-J-innen
5,09.06.23,18:30,B-J-innen
6,16.06.23,18:30,B-J-innen


In [103]:
#Club und Logo auslesen (befindet sich in jeder 3. Zeile ab 3. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    club1 = list()
    club2 = list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        #print(rows[i].text)
        removedSpace = re.sub(r'\n', '', rows[i].text)
        removedSpace = re.sub(r'\t', '', removedSpace)
        removedSpace = re.sub(r':', ';', removedSpace)
        splitted = removedSpace.split(';')
        club1.append(splitted[0])
        club2.append(splitted[1][0:-1])

df.insert(len(df.columns), "Heim", club1)
df.insert(len(df.columns), "Gast", club2)

In [104]:
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    logos_home = list()
    logos_guest= list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        logo = rows[i].find_all('div', class_="club-logo") #rows[i].text
        
        logo_link = re.findall(r'//(\S+)\">', str(logo))
        logos_home.append(logo_link[0])
        logos_guest.append(logo_link[1])
        
#found = re.search('AAA(.+?)ZZZ', string).group(1)        
df.insert(len(df.columns), 'KW', '')


df.insert(len(df.columns), 'Logo Heim', logos_home)
df.insert(len(df.columns), 'Logo Gast', logos_guest)

In [105]:
df

,Datum,Zeit,Team,Heim,Gast,KW,Logo Heim,Logo Gast
0,21.04.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,SGM Spfr Widdern,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
1,28.04.23,18:30,B-J-innen,SV Sülzbach,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
2,05.05.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,SGM TSV Talheim/​Heilbronn,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
3,12.05.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,TGV Dürrenzimmern,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
4,19.05.23,18:30,B-J-innen,TSV Langenbeutingen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
5,09.06.23,18:30,B-J-innen,SGM Spfr Widdern,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
6,16.06.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,SV Sülzbach,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...


In [106]:
links = table.find_all('a')
homeLinks = list();
guestLinks = list();
spielLinks = list();
for i in range(2, len(links),6):
    homeLinks.append(links[i].get('href'))
    guestLinks.append(links[i+1].get('href'))
    spielLinks.append(links[i+2].get('href'))
df.insert(len(df.columns), "home_link", homeLinks)
df.insert(len(df.columns), 'guest_link', guestLinks)
df.insert(len(df.columns), 'Spiel', spielLinks)
df

,Datum,Zeit,Team,Heim,Gast,KW,Logo Heim,Logo Gast,home_link,guest_link,Spiel
0,21.04.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,SGM Spfr Widdern,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/sgm-spfr-wi...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-b...
1,28.04.23,18:30,B-J-innen,SV Sülzbach,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sv-suelzbac...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/spiel/sv-suelzbach-i-s...
2,05.05.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,SGM TSV Talheim/​Heilbronn,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/sgm-tsv-tal...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-b...
3,12.05.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,TGV Dürrenzimmern,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/tgv-duerren...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-b...
4,19.05.23,18:30,B-J-innen,TSV Langenbeutingen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/tsv-langenb...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/spiel/tsv-langenbeutin...
5,09.06.23,18:30,B-J-innen,SGM Spfr Widdern,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-spfr-wi...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/spiel/sgm-spfr-widdern...
6,16.06.23,18:30,B-J-innen,SGM SC Ilsfeld/​Beilstein/​Abstatt ABI,SV Sülzbach,,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-ilsf...,https://www.fussball.de/mannschaft/sv-suelzbac...,https://www.fussball.de/spiel/sgm-sc-ilsfeld-b...


In [107]:
#merge columns to table
if (idx==0):
    df.to_csv('out.csv', index=False)
else:
    df.to_csv('out.csv', mode='a', index=False, header=False)

